In [17]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import chess.pgn
import chess.engine
import chess
import re
from hashlib import sha1


In [18]:
pgn = open("../../ccc/ccc-data/tournament-352729.pgn", encoding="utf-8")

In [23]:

game = chess.pgn.read_game(pgn)

In [20]:
str(game.mainline())

'1. Nf3 { book } 1... g6 { book } 2. d4 { book } 2... Bg7 { book } 3. g3 { book } 3... d6 { book } 4. Bg2 { book } 4... c6 { book } 5. b3 { book } 5... b5 { book } 6. e4 { book } 6... Bb7 { book } 7. O-O { book } 7... Nd7 { book } 8. Bb2 { book } 8... e6 { book } 9. e5 { +1.12/33 9.8s, ev=1.12, d=33, pd=Nh6, mt=00:00:09, tl=00:02:52, s=134394 kN/s, n=1315054777, pv=e5 Nh6 Nc3 d5 Re1 O-O h4 Re8 Bh3 Nf5 Ne2 c5 Bxf5 exf5 Nf4 Qc8 a4 b4 c3 a5 cxb4 axb4 Qd2 Qc6 Kh2 Bh6 Ng5 Bf8 Nxf7 c4 bxc4 dxc4 d5 Qc7 Rac1 c3 Bxc3 bxc3 Rxc3 Qa5 e6 Nf6 d6 Rec8 Rec1 Rxc3 Qxc3 Qxc3 Rxc3 Rxa4 Rc1 Rd4, tb=0, R50=50, wv=1.12 } 9... Nh6 { -0.96/16 14s, ev=-0.96, d=16, pd=Nc3, mt=00:00:14, tl=00:02:47, s=28 kN/s, n=409413, pv=Nh6 Nc3 d5 b4 a5 a3 Qe7 Qe1 Nb6 Ne2 Nc4 Bc3 a4 Bd2 Nf5 Bg5 Qc7 g4 Ne7 Nd2 Nxd2 Qxd2 h6 Bh4 h5 Bxe7 Qxe7 g5 Bf8 Rfe1 O-O-O h4 Kb8 Nf4 Bc8 Re3, tb=0, R50=50, wv=0.96 } 10. Nc3 { +1.22/33 4.7s, ev=1.22, d=33, pd=d5, mt=00:00:04, tl=00:02:49, s=163379 kN/s, n=759059437, pv=Nc3 d5 b4 a5 a3 Qe7 Ne2 a

In [24]:
sha1(str(game.mainline()).encode('utf-8')).hexdigest()

'd73e07b1841246cf4f18c982d9088b037a2d1bf0'

In [4]:
board = game.board()
seq = ""
turns = ['<b>', '<w>']
game_info = game

game_data = []
game_id = 1
for move in game.mainline_moves():
    game_info = game_info.next()
    comment = game_info.comment
    bool_turn = board.turn
    turn = turns[int(bool_turn)]
    lan = board.lan(move)
    move_num = board.fullmove_number
    pv_seq = ""
    pv, wv = None, None
    seq += turn
    if comment != 'book':
        pv = re.search(r"pv=(.+?),", comment).group(1).split(" ")
        wv = re.search(r"wv=(.+)", comment).group(1)
        for pv_move in pv:
            pv_move = board.parse_san(pv_move)
            pv_seq += turns[int(board.turn)] + board.lan(pv_move)
            board.push(pv_move)
        board = game_info.board()
        row_dict = {
            'id' : game_id,
            'move_num' : move_num,
            'turn' : turn,
            'seq_in' : seq,
            'seq_out': pv_seq[3:],
            'eval': wv
        }
        seq += lan
        game_data.append(row_dict)
    else:
        seq += lan
        board.push(move)



In [5]:
game_data

[{'id': 1,
  'move_num': 9,
  'turn': '<w>',
  'seq_in': '<w>Ng1-f3<b>g7-g6<w>d2-d4<b>Bf8-g7<w>g2-g3<b>d7-d6<w>Bf1-g2<b>c7-c6<w>b2-b3<b>b7-b5<w>e2-e4<b>Bc8-b7<w>O-O<b>Nb8-d7<w>Bc1-b2<b>e7-e6<w>',
  'seq_out': 'e4-e5<b>Ng8-h6<w>Nb1-c3<b>d6-d5<w>Rf1-e1<b>O-O<w>h2-h4<b>Rf8-e8<w>Bg2-h3<b>Nh6-f5<w>Nc3-e2<b>c6-c5<w>Bh3xf5<b>e6xf5<w>Ne2-f4<b>Qd8-c8<w>a2-a4<b>b5-b4<w>c2-c3<b>a7-a5<w>c3xb4<b>a5xb4<w>Qd1-d2<b>Qc8-c6<w>Kg1-h2<b>Bg7-h6<w>Nf3-g5<b>Bh6-f8<w>Ng5xf7<b>c5-c4<w>b3xc4<b>d5xc4<w>d4-d5<b>Qc6-c7<w>Ra1-c1<b>c4-c3<w>Bb2xc3<b>b4xc3<w>Rc1xc3<b>Qc7-a5<w>e5-e6<b>Nd7-f6<w>d5-d6<b>Re8-c8<w>Re1-c1<b>Rc8xc3<w>Qd2xc3<b>Qa5xc3<w>Rc1xc3<b>Ra8xa4<w>Rc3-c1<b>Ra4-d4',
  'eval': '1.12'},
 {'id': 1,
  'move_num': 9,
  'turn': '<b>',
  'seq_in': '<w>Ng1-f3<b>g7-g6<w>d2-d4<b>Bf8-g7<w>g2-g3<b>d7-d6<w>Bf1-g2<b>c7-c6<w>b2-b3<b>b7-b5<w>e2-e4<b>Bc8-b7<w>O-O<b>Nb8-d7<w>Bc1-b2<b>e7-e6<w>e4-e5<b>',
  'seq_out': 'Ng8-h6<w>Nb1-c3<b>d6-d5<w>b3-b4<b>a7-a5<w>a2-a3<b>Qd8-e7<w>Qd1-e1<b>Nd7-b6<w>Nc3-e2<b>Nb6-c4<w>Bb2-c3<b>a5-

In [ ]:
board = game.board()

move = list(game.mainline_moves())[0]

In [ ]:
board.push(move)

In [ ]:
board.lan()

In [4]:
from pyarrow import json
from pyarrow.json import ReadOptions
from pyarrow import parquet
import os

ro = ReadOptions(block_size=10<<20)

In [5]:
for fname in os.listdir("../outccc"):
    game_data = json.read_json(f"../outccc/{fname}", read_options=ro)
    parquet.write_table(game_data, f"../output_par/{os.path.splitext(fname)[0]}.parquet")